In [3]:
! export JAX_PLATFORMS=cpu

/gpfs/cfel/group/cmi/common/psi4/psi4conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


# Rovibrational matrix elements of Cartesian tensor operators

Will be computing matrix elements of spin-rotation tensors on $\text{H}_1$ and $\text{H}_2$ atoms of $\text{H}_2\text{S}$ as well as electric dipole-moment operator in the basis of rovibrational states.

In [4]:
from typing import Dict, List

import h5py
import numpy as np
from jax import config
from jax import numpy as jnp

from rovib.cartens import CART_IND
from rovib.spinrot_xy2 import spinrot_xy2
from rovib.symtop import threej_wang

config.update("jax_enable_x64", True)

First, read the indices of rotational cluster states for different $J$ quanta, sored in file `h2s_cluster_states.txt`. The rotational cluster states were identified in `h2s_clusters.ipynb`.

In [5]:
state_id_list = {}
with open("h2s_cluster_states_id.txt", "r") as fl:
    for line in fl:
        w = line.split()
        j = [int(w[0 + i * 12]) for i in range(4)]
        id = [int(w[1 + i * 12]) for i in range(4)]
        sym = [w[3 + i * 12] for i in range(4)]
        assert all(j[0] == elem for elem in j), f"Not all J quanta are equal: {j}"
        j_ = j[0]
        if j_ >= 50:
            state_id_list[j_] = {}
            for sym_, id_ in zip(sym, id):
                try:
                    state_id_list[j_][sym_].append(id_)
                except KeyError:
                    state_id_list[j_][sym_] = [id_]
            print(f"J = {j_}, cluster-state IDs: {state_id_list[j_]}")

J = 50, cluster-state IDs: {'B2': [277], 'A2': [242], 'B1': [242], 'A1': [280]}
J = 51, cluster-state IDs: {'A1': [267], 'B1': [299], 'B2': [270], 'A2': [299]}
J = 52, cluster-state IDs: {'B2': [324], 'A2': [291], 'A1': [327], 'B1': [291]}
J = 53, cluster-state IDs: {'A1': [316], 'B1': [354], 'B2': [318], 'A2': [353]}
J = 54, cluster-state IDs: {'B2': [385], 'A2': [345], 'A1': [387], 'B1': [344]}
J = 55, cluster-state IDs: {'A1': [371], 'B1': [413], 'A2': [412], 'B2': [373]}
J = 56, cluster-state IDs: {'B2': [447], 'A2': [399], 'B1': [399], 'A1': [450]}
J = 57, cluster-state IDs: {'A1': [432], 'B1': [487], 'B2': [434], 'A2': [486]}
J = 58, cluster-state IDs: {'B2': [520], 'B1': [468], 'A1': [523], 'A2': [469]}
J = 59, cluster-state IDs: {'A1': [503], 'B1': [565], 'A2': [563], 'B2': [505]}
J = 60, cluster-state IDs: {'B1': [542], 'A1': [600], 'A2': [544], 'B2': [599]}


Second, compute rovibrational matrix elements of spin-rotation tensors on $\text{H}_1$ and $\text{H}_2$ atoms for selected (cluster) states.

$$
M_{\omega,n}^{(J',l',J,l)}=\sum_{v',k'}\sum_{v,k}c_{v',k'}^{(J',\Gamma',l')*}c_{v,k}^{(J,\Gamma,l)}(-1)^{k'}\sum_{\sigma=-\omega}^{\omega}\sum_{\alpha,\beta=x,y,z}
\left(\begin{array}{ccc}
J & \omega & J' \\
k & \sigma & -k'\\
\end{array}\right)
U_{\omega\sigma,\alpha\beta}^{(\Omega)}\langle v'|\bar{M}_{\alpha\beta,n}|v\rangle.
$$

Here, $\Omega=2$ is the rank of spin-rotation tensor, $\omega=0..\Omega$, $n=1$ and 2 for $\text{H}_1$ and $\text{H}_2$, respectively,
$\bar{M}_{\alpha\beta,n}$ is the electronic spin-rotation tensor (see `h2s_spinrot.py` for details), $v$ and $k$ denote the vibrational and rotational quanta,
$J$ and $\Gamma$ denote 'good' quantum number of the rotational angular momentum and state symmetry, and $l$ is the state running index within
a set of states with the same $J$ and $\Gamma$.
The state expansion coefficients $c_{v,k}^{(J,\Gamma,l)}$ are computed in `h2s_rovib.ipynb` and stored for different values of $J$ quanta in files.
The matrix $U_{\omega\sigma,\alpha\beta}^{(\Omega)}$ transforms the Cartesian tensor of rank $\Omega$ into its spherical-tensor representation
(see `cartens.UMAT_CART_TO_SPHER`).

The rotational part of the expression
$$
R_{\omega,k',k,\alpha\beta}^{(J',J)} = \sum_{\sigma=-\omega}^{\omega}
\left(\begin{array}{ccc}
J & \omega & J' \\
k & \sigma & -k'\\
\end{array}\right)
U_{\omega\sigma,\alpha\beta}^{(\Omega)}
$$
is computed by the `symtop.threej_wang` function and transformed from symmetric-top basis $|J,k\rangle$ ($k=-J..J$) into the Wang's basis $|J,k,\tau\rangle$ ($k=0..J$, $\tau=0,1$ is partity as $(-1)^\tau$).

The vibrational matrix elements
$$
V_{n,v',v,\alpha\beta} = \langle v'|\bar{M}_{\alpha\beta,n}|v\rangle
$$
are computed in `h2s_rovib.ipynb` and stored in file.

Define to functions to load the energies and quantum numbers for selected states (`rovib_states`) and to compute the above matrix elements (`tensor_rovib_me`).

In [6]:
def rovib_states(j: int, state_ind_list: Dict[str, List[int]] = None, pmax: int = 20):
    """Reads rovibrational energies and quanta stored in files for different values
    of the J quantum number (the files are created in `h2s_rovib.ipynb`).

    Optionally, `state_ind_list[symmetry][<list of state indices>]` can be used to select specific states
    by their indices for different symmetries.
    """
    h5 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j}.h5", "r")
    energies = {}
    quanta = {}
    for sym in h5["energies"].keys():
        enr = h5["energies"][sym][:]
        coefs = h5["coefficients"][sym][:]
        vind = h5["vib-indices"][sym][:]
        rind = h5["rot-indices"][sym][:]
        qua = np.array(
            [elem[0].decode("utf-8").split(",") for elem in h5["quanta"][sym][:]]
        )
        if state_ind_list is not None and sym in state_ind_list:
            energies[sym] = enr[state_ind_list[sym]]
            quanta[sym] = qua[state_ind_list[sym]]
        else:
            energies[sym] = enr
            quanta[sym] = qua
    return energies, quanta

In [7]:
def tensor_rovib_me(
    rank: int,
    j1: int,
    j2: int,
    vib_me: np.ndarray,
    state_ind_list1: Dict[str, List[int]] = None,
    state_ind_list2: Dict[str, List[int]] = None,
    pmax: int = 20,
    linear: bool = False,
    tol: float = 1e-12,
):
    """Computes rovibrational matrix elements of a Cartesian tensor operator.

    This function calculates the rovibrational matrix elements using wavefunctions stored in
    separate files for different values of the J quantum number (the files are creaed in `h2s_rovib.ipynb`).
    The computation is performed for the specified bra (`j1`) and ket (`j2`) values of the J
    quantum number.

    Optionally, `state_ind_list1[symmetry][<list of state indices>]` and `state_ind_list2[...]`
    can be used to select specific bra and ket states by their indices for different symmetries.
    """

    # determine the order of Cartesian indices in the Cartesian-to-spherical tensor
    #   transformation matrix (in cartens.CART_IND and symtop.threej_wang)
    cart_ind = [["xyz".index(x) for x in elem] for elem in CART_IND[rank]]

    # reshape vibrational matrix elements such that the order of Cartesian indices
    #   correspond to the order in symtop.threej_wang output
    vib_me2 = jnp.moveaxis(
        jnp.array([vib_me[:, :, i, j] for (i, j) in cart_ind]), 0, -1
    )

    # compute rotational matrix elements of three-j symbol contracted with
    #   Cartesian-to-spherical tensor transformation matrix
    jktau_list1, jktau_list2, rot_me = threej_wang(rank, j1, j2, linear=linear)
    # rot_me[omega].shape = (2*j1+1, 2*j2+1, ncart)

    h5_1 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j1}.h5", "r")
    h5_2 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j2}.h5", "r")

    res = {}

    for sym1 in h5_1["energies"].keys():
        enr1 = h5_1["energies"][sym1][:]
        coefs1 = h5_1["coefficients"][sym1][:]
        vind1 = h5_1["vib-indices"][sym1][:]
        rind1 = h5_1["rot-indices"][sym1][:]
        qua1 = np.array(
            [elem[0].decode("utf-8").split(",") for elem in h5_1["quanta"][sym1][:]]
        )

        if state_ind_list1 is not None:
            if sym1 in state_ind_list1:
                ind = state_ind_list1[sym1]
                enr1 = enr1[ind]
                coefs1 = coefs1[:, ind]
            else:
                continue

        for sym2 in h5_2["energies"].keys():
            enr2 = h5_2["energies"][sym2][:]
            coefs2 = h5_2["coefficients"][sym2][:]
            vind2 = h5_2["vib-indices"][sym2][:]
            rind2 = h5_2["rot-indices"][sym2][:]
            qua2 = np.array(
                [elem[0].decode("utf-8").split(",") for elem in h5_2["quanta"][sym2][:]]
            )

            if state_ind_list2 is not None:
                if sym2 in state_ind_list2:
                    ind = state_ind_list2[sym2]
                    enr2 = enr2[ind]
                    coefs2 = coefs2[:, ind]
                else:
                    continue

            vib_me_ = vib_me2[jnp.ix_(vind1, vind2)]

            me = []
            for omega in range(rank + 1):
                me_ = jnp.einsum(
                    "ijc,ijc->ij", vib_me_, rot_me[omega][jnp.ix_(rind1, rind2)]
                )
                me.append(jnp.einsum("ik,ij,jl->kl", jnp.conj(coefs1), me_, coefs2))

            me = jnp.moveaxis(jnp.array(me), 0, -1)
            me = jnp.where(jnp.abs(me) < tol, 0.0, me)
            if jnp.count_nonzero(me) > 0:
                res[(sym1, sym2)] = jnp.array(me)

    return res

Using the functions above, compute matrix elements of spin-rotation tensors for selected rotational cluster states and store them in files.

In [8]:
pmax = 20
rank = 2

# read vibrational matrix elements of spin-rotation tensors

with h5py.File(f"h2s_vibme_pmax{pmax}.h5", "r") as h5:
    sr_h1_vib = h5["spin-rotation"]["h1"][:]
    sr_h2_vib = h5["spin-rotation"]["h2"][:]

# compute rovibrational matrix elements and store them in file

enr0, _ = rovib_states(0)
zpe = np.min([np.min(elem) for elem in enr0.values()])
print("zero-point energy:", zpe)

with h5py.File("h2s_cluster_states_me.h5", "w") as h5:
    h5_enr = h5.create_group("energies")
    h5_sr = h5.create_group("spin-rotation")
    h5_sr1 = h5_sr.create_group("h1")
    h5_sr2 = h5_sr.create_group("h2")

    for J in state_id_list.keys():
        print(f"\ncompute matrix elements for J = {J}")
        print("state indices:", state_id_list[J])

        enr, qua = rovib_states(J, state_ind_list=state_id_list[J])
        print("state energies - zpe:", [(key, val - zpe) for key, val in enr.items()])
        for sym in enr.keys():
            h5_enr.create_dataset(f"{J}/{sym}", data=enr[sym])

        sr1_me = tensor_rovib_me(
            rank, J, J, sr_h1_vib, state_id_list[J], state_id_list[J]
        )
        print("H1 spin-rotation matrix elements:")
        for sym1, sym2 in sr1_me:
            print(sym1, sym2, sr1_me[(sym1, sym2)])
            h5_sr1.create_dataset(f"{J}/{sym1}/{sym2}", data=sr1_me[(sym1, sym2)])

        sr2_me = tensor_rovib_me(
            rank, J, J, sr_h2_vib, state_id_list[J], state_id_list[J]
        )
        print("H2 spin-rotation matrix elements:")
        for sym1, sym2 in sr2_me:
            print(sym1, sym2, sr2_me[(sym1, sym2)])
            h5_sr2.create_dataset(f"{J}/{sym1}/{sym2}", data=sr2_me[(sym1, sym2)])

zero-point energy: 3291.1602143624377

compute matrix elements for J = 50
state indices: {'B2': [277], 'A2': [242], 'B1': [242], 'A1': [280]}
state energies - zpe: [('A1', array([21417.6586857])), ('A2', array([21417.65858143])), ('B1', array([21417.65868569])), ('B2', array([21417.65858143]))]
H1 spin-rotation matrix elements:
A1 A1 [[[-2.73188857+0.j  0.        +0.j -0.00641101+0.j]]]
A1 B2 [[[ 0.00000000e+00+0.j  3.50322401e-08+0.j -2.21833679e+00+0.j]]]
A2 A2 [[[-2.73188857+0.j  0.        +0.j -0.0064111 +0.j]]]
A2 B1 [[[0.00000000e+00+0.j 3.50296056e-08+0.j 2.21833679e+00+0.j]]]
B1 A2 [[[ 0.00000000e+00+0.j -3.50296056e-08+0.j  2.21833679e+00+0.j]]]
B1 B1 [[[-2.73188857+0.j  0.        +0.j -0.00641101+0.j]]]
B2 A1 [[[ 0.00000000e+00+0.j -3.50322401e-08+0.j -2.21833679e+00+0.j]]]
B2 B2 [[[-2.73188857+0.j  0.        +0.j -0.0064111 +0.j]]]
H2 spin-rotation matrix elements:
A1 A1 [[[-2.73188857+0.j  0.        +0.j -0.00641101+0.j]]]
A1 B2 [[[ 0.00000000e+00+0.j -3.50322401e-08+0.j  2

In [8]:
f = 1

enr, vec, qua = spinrot_xy2(
    f,
    rovib_enr,
    rovib_qua,
    rovib_sr1_me,
    rovib_sr2_me,
    spin_states=((0, "B2"), (1, "A1")),
    spins=(0.5, 0.5)
)
for key, val in enr.items():
    print(key, val.shape, val)

B1 (1517,) [ 3304.90655447  3306.25036378  3346.32166991 ... 28095.81400952
 28152.60879763 38855.620538  ]
B2 (1452,) [ 3310.53577666  3329.45817342  4493.67391267 ... 28124.183249
 38857.87783848 38876.34592702]


In [9]:
zpe = np.min([np.min(elem_) for elem in rovib_enr.values() for elem_ in elem.values()])
print("zpe:", zpe)

# print(rovib_enr[1]['B1']-zpe)
print(enr['B1']-zpe)


zpe: 3291.1602143624377
[1.37463401e+01 1.50901494e+01 5.51614555e+01 ... 2.48046538e+04
 2.48614486e+04 3.55644603e+04]


In [12]:
for sym, q in qua.items():
    print(sym, q)

B1 [(1, 'A2', 0, 'B2'), (1, 'B1', 1, 'A1'), (2, 'B1', 1, 'A1')]
B2 [(0, 'B2', 1, 'A1'), (1, 'A1', 0, 'B2'), (1, 'B2', 1, 'A1'), (2, 'B2', 1, 'A1')]
